In [ ]:
import fiftyone as fo
from detectron2.structures import BoxMode
import sys

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from matplotlib import pyplot as plt
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
import pandas as pd

In [ ]:
dataset = fo.Dataset.from_dir(
    dataset_dir="prepared",
    dataset_type=fo.types.FiftyOneDataset,
)


In [ ]:
def split_dataset(dataset):
        # Tag test images.
    testset_view = dataset.take(round(0.1 * len(dataset)), seed=42)



    testset_view.tag_samples("test")

    # Split remaining images into train and valid.
    nontestset_view = dataset.match_tags("test", bool=False)
    validset_view = nontestset_view.take(
        round(0.2 * len(nontestset_view)), seed=42
    )
    validset_view.tag_samples("valid")
    nontestset_view.match_tags("valid", bool=False).tag_samples("train")
    dataset.save()
    return dataset

In [ ]:
dataset.default_classes

In [ ]:
def get_fiftyone_dicts(dataset):
    dataset.compute_metadata()

    dataset_dicts = []
    for sample in dataset:
        height = sample.metadata["height"]
        width = sample.metadata["width"]
        record = {}
        record["file_name"] = sample.filepath
        record["image_id"] = sample.id
        record["height"] = height
        record["width"] = width

        objs = []
        for det in sample.ground_truth.detections:
            tlx, tly, w, h = det.bounding_box
            bbox = [int(tlx*width), int(tly*height), int(w*width), int(h*height)]

            obj = {
                "bbox": bbox,
                "bbox_mode": BoxMode.XYWH_ABS,
                "category_id": dataset.default_classes.index(det.label),
            }
            objs.append(obj)

        record["annotations"] = objs
        dataset_dicts.append(record)

    return dataset_dicts


In [ ]:
for d in ["train", "valid"]:
    view = dataset.match_tags(d)
    DatasetCatalog.register("fiftyone_" + d, lambda view=view: get_fiftyone_dicts(view))
    MetadataCatalog.get("fiftyone_" + d).thing_classes=['cocoa', 'invalid']

metadata = MetadataCatalog.get("fiftyone_train")


In [ ]:
print(metadata)

In [ ]:
dataset_dicts = get_fiftyone_dicts(dataset.match_tags("train"))
ids = [dd["image_id"] for dd in dataset_dicts]

view = dataset.select(ids)
session = fo.launch_app(view)

In [ ]:
session.freeze()  # screenshot the App

In [ ]:
label_count = view.count_values("ground_truth.detections.label")
print(label_count)

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.OUTPUT_DIR = "detectron_files/models"
#cfg.merge_from_file(model_zoo.get_config_file(""))
cfg.DATASETS.TRAIN = ("fiftyone_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.TEST.EVAL_PERIOD = 1000
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00001  # pick a good LR
cfg.SOLVER.MAX_ITER = 10000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = [] # do not decay learning rate
cfg.SOLVER.AMP.ENABLED = True
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64  # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # only has one class (Vehicle registration plate). (see https://detectron2.readthedocs.io/tutorials/datasets.

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
#html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
def detectron_to_fo(outputs, img_w, img_h):
    # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    detections = []
    instances = outputs["instances"].to("cpu")
    for pred_box, score, c in zip(
        instances.pred_boxes, instances.scores, instances.pred_classes,
    ):
        x1, y1, x2, y2 = pred_box
        bbox = [float(x1)/img_w, float(y1)/img_h, float(x2-x1)/img_w, float(y2-y1)/img_h]
        detection = fo.Detection(label=dataset.default_classes[c], confidence=float(score), bounding_box=bbox, )
        detections.append(detection)

    return fo.Detections(detections=detections)

In [ ]:
for sample in dataset:
    img_w = sample.metadata["width"]
    img_h = sample.metadata["height"]
    img = cv2.imread(sample["filepath"])
    outputs = predictor(img)
    detections = detectron_to_fo(outputs, img_w, img_h)
    sample["predictions"] = detections
    sample.save()

In [ ]:
print(outputs['instances'].pred_classes)

In [ ]:
fo.Session(dataset)

In [ ]:
splits = ['train','valid','test']
for split_tag in splits:
    view = dataset.match_tags([split_tag])

    # Evaluate the objects in the `predictions`
    # field with respect to the
    # objects in the `ground_truth` field
    eval_key = f"eval_predictions_{split_tag}"
    results = view.evaluate_detections(
        "predictions",
        gt_field="ground_truth",
        eval_key=eval_key,
        compute_mAP=True,
        classes=dataset.default_classes,
        missing="background",
        classwise=True,
    )
    # whether to consider objects with different label
    # values as always non-overlapping (True) or to compute IoUs
    # for all objects regardless of label (False)

    # the COCO mAP evaluator averages the mAP
    # over 10 IoU thresholds from 0.5 to 0.95
    # with a step size of 0.05 (AP@[0.5:0.05:0.95])
    # To be found in the source of fiftyone.
    # "https://github.com/voxel51/fiftyone/blob/"
    # "acf3a8f886505d852903e320d057057813261993/fiftyone/"
    # "utils/eval/coco.py#L91"
    mAP = results.mAP()
    print(f"mAP@[0.5:0.05:0.95] {split_tag} : " + str(mAP))
    classwise_ap_df = pd.DataFrame(
        columns=["Label", "AP@[0.5:0.05:0.95]"]
    )
    for label in dataset.default_classes:
        class_AP = results.mAP([label])
        print(
            f"AP@[0.5:0.05:0.95] of {split_tag} ({label}): "
            + str(class_AP)
        )
        classwise_ap_df = classwise_ap_df._append(
            {"Label": label, "AP@[0.5:0.05:0.95]": class_AP},
            ignore_index=True,
        )


    results.print_report()
    report = results.report()
    weighted_avg_precision = report["weighted avg"]["precision"]
    weighted_avg_recall = report["weighted avg"]["recall"]


    # Print some statistics about the total TP/FP/FN counts
    mean_tp = view.sum(f"{eval_key}_tp")
    mean_fp = view.sum(f"{eval_key}_fp")
    mean_fn = view.sum(f"{eval_key}_fn")
    print(f"TP ({split_tag}): {mean_tp}")
    print(f"FP ({split_tag}): {mean_fp}")
    print(f"FN ({split_tag}): {mean_fn}")


    # class_counts = view.count_values("predictions.detections.label")

    # pr_curve_path = os.path.join(
    #     artifacts_path, f"PR_curve_{split_tag}.png"
    # )
    # pr_curve_plot: Figure = results.plot_pr_curves(
    #     classes=list(class_counts.keys()),
    #     backend="matplotlib",
    #     style="dark_background",
    # )
    # pr_curve_plot.savefig(pr_curve_path, dpi=250)
    # mlflow.log_artifact(pr_curve_path)

    # conf_mat_path = os.path.join(
    #     self.artifacts_path, f"confusion_matrix_{split_tag}.png"
    # )
    # conf_mat_plot: Figure = results.plot_confusion_matrix(
    #     backend="matplotlib"
    # )
    # conf_mat_plot.savefig(conf_mat_path, dpi=250)
    # mlflow.log_artifact(conf_mat_path)

    # mlflow.end_run()

    # return dataset
